In [1]:
!pip install dspy

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------------------------- ------ 1.6/1.9 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 7.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.0
    Uninstalling joblib-1.4.0:
      Successfully uninstalled joblib-1.4.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled d

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.62 requires langchain-community<0.3.0,>=0.2.6, but you have langchain-community 0.2.4 which is incompatible.
together 1.2.3 requires pydantic<3.0.0,>=2.6.3, but you have pydantic 2.5.0 which is incompatible.


In [7]:
import dspy

# Define the task
class QA(dspy.Signature):
    """Answer questions based on the given context."""
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField()

# # Create a basic model
# lm = dspy.OpenAILanguageModel(model="gpt-3.5-turbo")
lm=dspy.OllamaLocal(model="llama3:8b-instruct-q4_K_M")

# lm = dspy.{provider_listed_below}(model="your model", model_request_kwargs="...")
# Define the prompt


In [9]:
class BasicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.qa = QA()

    def forward(self, context, question):
        return self.qa(context=context, question=question)

# Create a dataset for optimization
train_data = [
    {"context": "The capital of France is Paris.", "question": "What is the capital of France?", "answer": "Paris"},
    {"context": "Python is a programming language.", "question": "What is Python?", "answer": "A programming language"},
    # Add more examples...
]

# Define the metric for evaluation
def exact_match(gold, pred):
    return gold.lower() == pred.lower()

# Create the optimizer
optimizer = dspy.Optimize(
    metric=exact_match,
    module=BasicQA(),
    trainset=train_data,
    valset=train_data,  # Using the same set for simplicity
    optimizer="ollama",
    max_rounds=5
)

# Run the optimization
optimized_qa = optimizer.run()

# Test the optimized model
context = "The Eiffel Tower is located in Paris, France."
question = "Where is the Eiffel Tower located?"

result = optimized_qa(context=context, question=question)
print(f"Optimized model answer: {result.answer}")

AttributeError: module 'dspy' has no attribute 'Optimize'

In [2]:
!pip install llama_cpp

ERROR: Could not find a version that satisfies the requirement llama_cpp (from versions: none)
ERROR: No matching distribution found for llama_cpp


In [3]:
import dspy
import ollama

class OllamaModel(dspy.LM):
    def __init__(self, model_name="llama2"):
        self.model_name = model_name

    def basic_request(self, prompt, **kwargs):
        response = ollama.generate(model=self.model_name, prompt=prompt)
        return response['response']

    def __call__(self, prompt, **kwargs):
        response = self.basic_request(prompt, **kwargs)
        return dspy.Completion(text=response, logprob=0)  # Ollama doesn't provide logprobs, so we use 0




AttributeError: module 'dspy' has no attribute 'LM'

In [ ]:
# Use Ollama to run the LLaMA model
lm = OllamaModel(model_name="llama3:8b-instruct-q4_K_M")  # or whatever LLaMA model you have in Ollama

# Define the task (same as before)
class QA(dspy.Signature):
    """Answer questions based on the given context."""
    context = dspy.InputField()
    question = dspy.InputField()
    answer = dspy.OutputField()

# Define the prompt (same as before)
class BasicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.qa = QA()

    def forward(self, context, question):
        return self.qa(context=context, question=question)

# Create a dataset for optimization (same as before)
train_data = [
    {"context": "The capital of France is Paris.", "question": "What is the capital of France?", "answer": "Paris"},
    {"context": "Python is a programming language.", "question": "What is Python?", "answer": "A programming language"},
    # Add more examples...
]

# Define the metric for evaluation (same as before)
def exact_match(gold, pred):
    return gold.lower() == pred.lower()

# Create the optimizer
optimizer = dspy.Optimize(
    metric=exact_match,
    module=BasicQA(),
    trainset=train_data,
    valset=train_data,  # Using the same set for simplicity
    optimizer="rm",  # Use a different optimizer as "openai" might not work with local models
    max_rounds=5
)

# Run the optimization
optimized_qa = optimizer.run()

# Test the optimized model
context = "The Eiffel Tower is located in Paris, France."
question = "Where is the Eiffel Tower located?"

result = optimized_qa(context=context, question=question)
print(f"Optimized model answer: {result.answer}")

In [1]:
import dspy

# Define a simple language model
lm = dspy.OpenAI(model="gpt-3.5-turbo")

# Define a signature for our task
class Summarize(dspy.Signature):
    """Summarize the given text."""
    text = dspy.InputField()
    summary = dspy.OutputField()

# Create a basic module
summarizer = dspy.Module(Summarize)

# Example text to summarize
text = "DSPy is a framework for solving AI tasks using language models and retrieval models. It allows developers to write task-specific programs and optimize them automatically."

# Optimize the module
optimized_summarizer = dspy.ChainOfThought(summarizer)

# Use the optimized module
result = optimized_summarizer(text=text)
print(result.summary)

TypeError: Module.__init__() takes 1 positional argument but 2 were given

In [2]:
import dspy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a local model (for example, T5-base)
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define a signature for a simple Q&A task
class QuestionAnswer(dspy.Signature):
    question = dspy.InputField(desc="The question to be answered")
    answer = dspy.OutputField(desc="The model's answer")

# Create a DSPy module using the signature
qa_module = dspy.Predict(QuestionAnswer, model=model, tokenizer=tokenizer)

# Define an optimizer for the module to refine the prompt and model weights
optimizer = dspy.Optimizer(qa_module)

# Sample input
input_data = {"question": "What is the capital of France?"}

# Run optimization process
optimized_output = optimizer.optimize(input_data)

# Print the optimized answer
print("Optimized Answer:", optimized_output.answer)


c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MY IT STORE\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

AttributeError: module 'dspy' has no attribute 'Optimizer'

In [ ]:
import dspy

turbo=dspy.OllamaLocal(model="llama3.1:8b-instruct-q4_K_M")
dspy.settings.configure(lm=turbo)

In [4]:
import sys
import os
import dspy
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from rich import print

# turbo = dspy.OpenAI(model='gpt-3.5-turbo')

turbo=dspy.OllamaLocal(model="llama3.1:8b-instruct-q4_K_M",timeout_s=1000)
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

In [5]:
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [6]:
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print(len(trainset), len(devset))

train_example = trainset[0]
print(f"Question: {train_example.question}")
print(f"Answer: {train_example.answer}")

dev_example = devset[18]
print(f"Question: {dev_example.question}")
print(f"Answer: {dev_example.answer}")
print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")

print(f"For this dataset, training examples have input keys {train_example.inputs().keys()} and label keys {train_example.labels().keys()}")
print(f"For this dataset, dev examples have input keys {dev_example.inputs().keys()} and label keys {dev_example.labels().keys()}")


20 50

Question: At My Window was released by which American singer-songwriter?

Answer: John Townes Van Zandt

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Answer: English

Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}

For this dataset, training examples have input keys ['question'] and label keys ['answer']

For this dataset, dev examples have input keys ['question'] and label keys ['answer', 'gold_titles']

In [7]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

generate_answer = dspy.Predict(BasicQA)
pred = generate_answer(question=dev_example.question)
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

turbo.inspect_history(n=1)

generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)
pred = generate_answer_with_chain_of_thought(question=dev_example.question)
print(f"Question: {dev_example.question}")
print(f"Thought: {pred.rationale.split('.', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(dev_example.question).passages

print(f"Top {retrieve.k} passages for question: {dev_example.question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

print(retrieve("When was the first FIFA World Cup held?").passages[0])

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Predicted Answer: Robert Irvine





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: Robert Irvine





Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?

Thought:

...

Top 3 passages for question: What is the nationality of the chef and restaurateur featured in Restaurant: 
Impossible? 
 ------------------------------

1] Restaurant: Impossible | Restaurant: Impossible is an American reality television series, featuring chef and 
restaurateur Robert Irvine, that aired on Food Network from 2011 to 2016.

2] Jean Joho | Jean Joho is a French-American chef and restaurateur. He is chef/proprietor of Everest in Chicago 
(founded in 1986), Paris Club Bistro & Bar and Studio Paris in Chicago, The Eiffel Tower Restaurant in Las Vegas, 
and Brasserie JO in Boston.

3] List of Restaurant: Impossible episodes | This is the list of the episodes for the American cooking and reality 
television series "Restaurant Impossible", produced by Food Network. The premise of the series is that within two 
days and on a budget of $10,000, celebrity chef Robert Irvine renovates a failing American restaurant with the goal
of helping to restore it to profitability and prominence. Irvine is assisted by a designer (usually Taniya Nayak, 
Cheryl Torrenueva, or Lynn Keagan, but sometimes Vanessa De Leon, Krista Watterworth, Yvette Irene, or Nicole 
Faccuito), along with general contractor Tom Bury, who sometimes does double duty as both general contractor and 
designer. After assessing the problems with the restaurant, Robert Irvine typically creates a plan for the new 
decor, oversees the cleaning of the restaurant, reduces the size of the menu and improves the food, develops a 
promotional activity, educates the restaurant's owners, or trains the staff, as needed by each restaurant.

History of the FIFA World Cup | The FIFA World Cup was first held in 1930, when FIFA president Jules Rimet decided 
to stage an international football tournament. The inaugural edition, held in 1930, was contested as a final 
tournament of only thirteen teams invited by the organization. Since then, the World Cup has experienced successive
expansions and format remodeling to its current 32-team final tournament preceded by a two-year qualifying process,
involving over 200 teams from around the world.

In [8]:






class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

my_question = "What castle did David Gregory inherit?"
pred = compiled_rag(my_question)

print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

turbo.inspect_history(n=1)


Failed to run or to evaluate example Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000001EA8499D620> due to HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120).


Failed to run or to evaluate example Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000001EA8499D620> due to HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120).


 10%|█         | 2/20 [05:23<50:31, 168.43s/it]

Failed to run or to evaluate example Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000001EA8499D620> due to HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120).


Failed to run or to evaluate example Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x000001EA8499D620> due to HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120).


In [ ]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

# Evaluating the Answers
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))
    return gold_titles.issubset(found_titles)

compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)
        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

turbo.inspect_history(n=3)

def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False
    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]
    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False
    return True

teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=gold_passages_retrieved)
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=gold_passages_retrieved)

print(f"## Retrieval Score for RAG: {compiled_rag_retrieval_score}")
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

compiled_baleen("How many storeys are in the castle that David Gregory inherited?")
turbo.inspect_history(n=3)